### 叔叔我啊，要开始训练了捏

首先，获取模型和数据


In [1]:
import os

import numpy as np
import torch
from tqdm import tqdm

from dataset import MyDataset
from model import LSTM, MyLSTM, MyTransformerEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MyTransformerEncoder(d_model=100, output_dimension=1)
# model = MyLSTM(d_model=100, output_dimension=1)
# model = LSTM(256)
train_data = np.load("./npys/train/sentence_code.npy")
train_labels = np.load("./npys/train/labels.npy")
vocabulary_vectors = np.load("./npys/vocabulary_vectors.npy")
dataset = MyDataset(train_data[12000:13000], train_labels[12000:13000], vocabulary_vectors)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True) # Windows上，num_workers太高会慢，很sb


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.995)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [2]:
def train(max_epoch):
    loss_fn = torch.nn.BCELoss().to(device)
    # loss_fn = torch.nn.CrossEntropyLoss().to(device)
    for epoch in range(max_epoch):
        avg_loss = []
        for X, y in dataloader:
            optimizer.zero_grad()
            output = model(X)
            # print(X.shape)
            # print(output)
            # print(y)
            loss = loss_fn(output, y)
            avg_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            scheduler.step()
        if epoch%10==0:
            print("Epoch: %d | loss: %f"%(epoch, np.average(avg_loss)))

训练，保存：

In [3]:
train(200)
os.makedirs("./trained_models/", exist_ok=True)
torch.save(model, "./trained_models/transformer%d"%200)

Epoch: 0 | loss: 0.713217
Epoch: 10 | loss: 0.641049
Epoch: 20 | loss: 0.311088
Epoch: 30 | loss: 0.236008
Epoch: 40 | loss: 0.207907
Epoch: 50 | loss: 0.181901
Epoch: 60 | loss: 0.188094
Epoch: 70 | loss: 0.155537
Epoch: 80 | loss: 0.160156
Epoch: 90 | loss: 0.137808
Epoch: 100 | loss: 0.141360
Epoch: 110 | loss: 0.140189
Epoch: 120 | loss: 0.142932
Epoch: 130 | loss: 0.142101
Epoch: 140 | loss: 0.125473
Epoch: 150 | loss: 0.137950
Epoch: 160 | loss: 0.119794
Epoch: 170 | loss: 0.122597
Epoch: 180 | loss: 0.130121
Epoch: 190 | loss: 0.127018
